In [1]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.make_yearly_files import make_yearly_files
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Multigrams for Training Word-Embedding Models**

## **Goal**: Download and preprocess mulitgrams for use in training `word2vec` models. 

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

**NOTE:** You'll probably want to have run `workflow_unigrams.ipynb` before processing multigrams. That workflos allows you create a vocabulary file for filtering out uncommon tokens from the multigrams. Although you can run the `filter_ngrams` module without a vocab file, most use cases will call for one.

### Download multigrams
Here, I'm using `download_ngrams` module to fetch 5grams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). You may wish to specify `compress=True` becausae 5gram files are _big_.

In [2]:
download_ngram_files(
    ngram_size=5,
    ngram_type='tagged',
    repo_release_id='20200217',
    repo_corpus_id='eng-fiction',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True
)

Start Time:                2025-02-10 17:28:28.189636

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/eng-fiction-5-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/1download
File index range:          0 to 1448
File URLs available:       1449
File URLs to use:          1449
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/5-00000-of-01449.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/5-01448-of-01449.gz
Ngram size:                5
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  True



Downloading:   0%|          | 0/1449 [00:00<?, ?files/s]


End Time:                  2025-02-10 17:51:39.915662
Total runtime:             0:23:11.726026



### Convert files from TXT to JSONL
This module converts the original multigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

In [3]:
convert_to_jsonl_files(
    ngram_size=5,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-10 17:51:39.919324

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/2convert
File index range:          0 to 1448
Files available:           1449
Files to use:              1449
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/1download/5-00000-of-01449.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/1download/5-01448-of-01449.txt.lz4
Ngram size:                5
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Converting:   0%|          | 0/1449 [00:00<?, ?files/s]


End Time:                  2025-02-10 17:59:46.573908
Total runtime:             0:08:06.654584



### Make multigrams all lowercase
This module lowercases all characters in the multigrams. Most use cases benefit from this.

In [4]:
lowercase_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-10 17:59:46.579368

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/3lowercase
File index range:          0 to 596
Files available:           597
Files to use:              597
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/2convert/5-00000-of-01449.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/2convert/5-01448-of-01449.jsonl.lz4
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lowercasing:   0%|          | 0/597 [00:00<?, ?files/s]


End Time:                  2025-02-10 18:03:42.868946
Total runtime:             0:03:56.289578



### Lemmatize the multigrams
Likewise, most use cases will benefit from multigrams that are lemmatized—that is, reduced to their base form. This requires POS-tagged multigrams. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. The POS tag will then be discarded as it is no longer useful.

In [5]:
lemmatize_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-10 18:03:42.875547

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/4lemmatize
File index range:          0 to 596
Files available:           597
Files to use:              597
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/3lowercase/5-00000-of-01449.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/3lowercase/5-01448-of-01449.jsonl.lz4
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lemmatizing:   0%|          | 0/597 [00:00<?, ?files/s]


End Time:                  2025-02-10 18:12:18.098022
Total runtime:             0:08:35.222475



### Filter the multigrams
This module removes tokens that provide little information about words' semantic context—specifically, those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`). You can also specify a **vocabulary file** like the one illustrated in the unigram workflow. A vocabulary file is simply a list of the _N_ most common words in the unigram corpus; the multigram tokens are checked against this list and those that don't appear in it are dropped.

The filtering process will inevitably turn some longer ngrams (e.g., 5grams) into shorter ones (e.g., 3grams) after unwanted tokens are dropped. The training of word-embedding models requires _linguistic context_—which in turn requires ngrams containing more than one token. A unigram isn't useful for helping a model learn what "company" a word keeps. Thus, the `min_tokens` option allows you to drop ngrams that fall below a specified length during filtering. If filtering results in an ngram with fewer than the minimum tokens, all data for that ngram is dropped entirely. Here, I've set `min_tokens=2`, since two tokens (and higher) provide at least some contextual information.

In [6]:
filter_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=2,
    vocab_file='1gram-corpus-vocab_list_match.txt',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                   2025-02-10 18:12:18.103413

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/5filter
File index range:             0 to 596
Files available:              597
Files to use:                 597
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/4lemmatize/5-00000-of-01449.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/4lemmatize/5-01448-of-01449.jsonl.lz4
Ngram size:                   5
Number of workers:            48
Compress output files:        True
Overwrite existing files:     True
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:    

Filtering:   0%|          | 0/597 [00:00<?, ?files/s]


Filtering Results (Dropped)
Stopword tokens:              0 
Short-word tokens:            0 
Tokens with numerals:         0 
Tokens with non-alpha chars:  0
Out-of-vocab tokens:          623721673
Entire ngrams:                58963042 

End Time:                  2025-02-10 18:16:20.830044
Total runtime:             0:04:02.726631



### Sort and combine the multigram files
This modules creates a single, fully-sorted multigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

Because this step can take a _very_ long time for larger multigrams (e.g., 5grams), we can run it in sessions using the `start_iteration` and `end_iteration` options. Iteration 1 comes after the initial file sort. If you only have time to complete, say, iterations 1–3, you can set `end_iteration=3`. During a later session, you can specify `start_iteration=4` to pick up where you left off.

In [7]:
sort_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    workers=12,
    sort_key='ngram',
    compress=True,
    overwrite=False,
    sort_order='ascending',
    delete_input=True
)

Start Time:                2025-02-10 18:16:28.980756

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/5gram-merged.jsonl.lz4
Files available:           593
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/5filter/5-00009-of-01449.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/5filter/5-01448-of-01449.jsonl.lz4
Files to use:              593
Ngram size:                5
Number of workers:         12
Compress output files:     True
Overwrite existing files:  False
Sort key:                  ngram
Sort order

Sorting:   0%|          | 0/593 [00:00<?, ?files/s]


Iteration 1: merging 593 files into 48 chunks using 48 workers.
  2 chunk(s) with 1 file(s)
  1 chunk(s) with 5 file(s)
  3 chunk(s) with 7 file(s)
  1 chunk(s) with 8 file(s)
  3 chunk(s) with 9 file(s)
  1 chunk(s) with 10 file(s)
  4 chunk(s) with 11 file(s)
  3 chunk(s) with 12 file(s)
  2 chunk(s) with 13 file(s)
  7 chunk(s) with 14 file(s)
  20 chunk(s) with 15 file(s)
  1 chunk(s) with 16 file(s)

Iteration 2: merging 48 files into 24 chunks using 24 workers.
  24 chunk(s) with 2 file(s)

Iteration 3: merging 24 files into 12 chunks using 12 workers.
  12 chunk(s) with 2 file(s)

Iteration 4: merging 12 files into 6 chunks using 6 workers.
  6 chunk(s) with 2 file(s)

Iteration 5: merging 6 files into 3 chunks using 3 workers.
  3 chunk(s) with 2 file(s)

Iteration 6: merging 3 files into 1 chunks using 1 workers.
  1 chunk(s) with 3 file(s)
Merging complete. Final file: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/5gram-merged.jsonl.lz4

End 

### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs True, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether sort_ngrams was run with compress=True.

In [8]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/'
        '5gram_files/6corpus/5gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

Lines: 144587522line [16:20, 147468.24line/s]

The file is sorted.


### Consolidate duplicate multigrams
This module consolidates the sorted multigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

In [9]:
consolidate_duplicate_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    lines_per_chunk=500000,
    compress=True,
    overwrite=True
)

Start Time:                2025-02-10 19:23:46.158741

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/5gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/5gram-corpus.jsonl.lz4
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/temp_chunks
Ngram size:                5
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Created and Sorted: 290 chunks
Merged: 290 chunks


End Time:                  2025-02-10 19:42:29.227367
Total runtime:             0:18:43.068626



### View line [OPTIONAL]
If we want, we can inspect a line in the file.

In [10]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '5gram_files/6corpus/5gram-corpus.jsonl.lz4'
    ),
    start_line=1650262,
    end_line=1650263,
    parse_json=True
)

Line 1650262: {'ngram': 'man historical emergence', 'freq_tot': 133, 'doc_tot': 124, 'freq': {'1997': 6, '1998': 2, '1999': 6, '2000': 5, '2001': 6, '2002': 2, '2003': 12, '2004': 6, '2005': 6, '2006': 3, '2007': 8, '2008': 4, '2009': 20, '2010': 8, '2011': 1, '2012': 5, '2013': 11, '2014': 12, '2015': 3, '2016': 4, '2017': 1, '2018': 2}, 'doc': {'1997': 4, '1998': 2, '1999': 6, '2000': 5, '2001': 4, '2002': 2, '2003': 11, '2004': 6, '2005': 6, '2006': 3, '2007': 5, '2008': 4, '2009': 20, '2010': 8, '2011': 1, '2012': 5, '2013': 10, '2014': 12, '2015': 3, '2016': 4, '2017': 1, '2018': 2}}
Line 1650263: {'ngram': 'man historical existence', 'freq_tot': 295, 'doc_tot': 290, 'freq': {'1935': 3, '1936': 1, '1938': 3, '1939': 5, '1941': 2, '1942': 3, '1945': 3, '1946': 3, '1947': 1, '1948': 1, '1951': 2, '1952': 3, '1953': 1, '1954': 7, '1955': 4, '1956': 1, '1958': 2, '1959': 1, '1960': 9, '1962': 5, '1963': 1, '1964': 10, '1965': 8, '1966': 8, '1967': 11, '1968': 7, '1969': 9, '1970': 4, 

### Make yearly files
This module converts the overall corpus file into yearly corpora. For each year in which an ngram appeared, a `<year>.jsonl` file (or `<year>.jsonl.lz4` if `compress=True`) will be created. Each line in a yearly file contains an ngram, a `freq` value (the number of times it appeared that year), and a `doc` value (the number of unique documents it appeared in that year).

I found it difficult to prevent memory exhaustion when processing 5grams with 128GB of RAM. Users may have to reduce the number of processors and/or the `chunk_size` to stay within their limits. Also note that the final clean-up step, in which many temporary files get deleted, can take several minutes to complete. 

After creating yearly corpora, we can proceed to train `word2vec` models as shown in the `workflow_train_models.ipynb` notebook.

In [2]:
make_yearly_files(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    overwrite=True,
    compress=True,
    workers=14,
    chunk_size=250000
)

Start Time:                2025-02-10 23:25:04.767529

Processing Info
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/5gram-corpus.jsonl.lz4
Yearly file directory:     /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data
Compress output files:     True
Number of workers:         14
Overwrite existing files:  True

Created and processed 139 chunks
Merged temp files for 388 years

End Time:                  2025-02-10 23:36:00.689636
Total runtime:             0:10:55.922107



### Next Steps
Now that you've created yearly corpora of multigrams, it's time to train word embeddings using `word2vec`. See the `workflow_train_models.ipynb` notebook for a guide to training and optimizing yearly word embeddings.